In [1]:
import pandas as pd
import re

In [2]:
def clean_census(file, year):
    
    column_names= ['Variable', 'Arbutus-Ridge', 'Downtown', 'Dunbar-Southlands',
               'Fairview', 'Grandview-Woodland', 'Hastings-Sunrise',
               'Kensington-Cedar Cottage', 'Kerrisdale', 'Killarney', 'Kitsilano',
               'Marpole', 'Mount Pleasant', 'Oakridge', 'Renfrew-Collingwood',
               'Riley Park', 'Shaughnessy', 'South Cambie', 'Strathcona',
               'Sunset', 'Victoria-Fraserview', 'West End', 'West Point Grey',
               'Vancouver CSD', 'Vancouver CMA']
   
    # read in csv file as dataframe
    df = pd.read_csv(file, encoding='latin-1', skiprows=4)
    
    # remove 'ID' column if present
    df.drop(columns='ID', inplace=True, errors='ignore')
    
    # rename columns 
    df.set_axis(column_names, axis=1, inplace=True)
    
    # remove empty rows
    df.dropna(0, 'all', inplace=True)
    
    # remove leading whitespace from variables
    df.Variable = df.Variable.apply(lambda x: (x.lstrip()).rstrip())
    df.drop(df[df.Variable.str.contains('20%.*data', flags=re.IGNORECASE) ].index , inplace=True)
    
    # convert all data to strings except NaN values
    df.iloc[:,1:25] = df.iloc[:,1:25].applymap(lambda x: str(x) if x == x else x)
    
    # convert data to float
    df.iloc[:,1:25] = df.iloc[:,1:25].applymap(lambda x: re.sub("[-]", "0", x) if x == x else x)
    df.iloc[:,1:25] = df.iloc[:,1:25].applymap(lambda x: float(re.sub("[,$]", "", x)) if x == x else x)
    
    # divide the census datasets into subgroups
    sub_dict = create_subgroup_dict(df, year)
    
    # clean the datatables by topics
    census_dict = clean_age(sub_dict, year)
    census_dict = clean_marital_status(census_dict, year)
    census_dict = clean_private_households(census_dict, year)
    census_dict = clean_couple_fam_structure(census_dict, year)
    census_dict = clean_language_detailed(census_dict, year)
    census_dict = clean_official_language(census_dict, year)
    census_dict = clean_structural_dwelling_type(census_dict, year)
    census_dict = clean_household_size(census_dict, year)
        
    return census_dict

In [3]:
def create_subgroup_dict(df, year):
    
    # separate dataframe by 'Variables' containing regex expressions:
    if year == 2001:
        re1 = ['total.*by', 
               'population.*by', 
               'common-law couples',
               '^Male', 
               '^Female', 
               'total - male', 
               'total - female']
        
    elif year == 2006:
        re1 = ['total.*by', 
               'population.*by',
               'common-law couples',
               '^Male[s\s,]', 
               '^Female[s\s,]',
              'total - mobility']
        
    elif year == 2011:
        df.drop(index=201 , inplace=True)
        re1 = ['total.*by', 
               'population.*by', 
               'common-law couples',
               'males',
               'Total population excluding institutional residents',  
               'Total.*in private households'] 
    else:
        re1 = ['^total', 'population.*by', 'males']
    
    subgroup = list(df[df.Variable.str.contains('|'.join(re1), flags=re.IGNORECASE)].index)
    subgroup.append(len(df.Variable)+1)
    subgroup = subgroup[1:]
    
    # create census dictionary of sub datasets
    # initialize variables for the lookup dictionary
    start = 0
    census_dict = {}
    
    for s in subgroup:
        sub_df = df.loc[start:s-1]

        # transpose dataframe and rename column
        sub_df = sub_df.set_index('Variable').T.reset_index().rename(columns={'index': 'LocalArea'})

        # clean up names and store dataframes into the dictionary
        census_dict[df.Variable[start].rstrip().lstrip()] = sub_df
        start = s
        
    return census_dict


In [4]:
def clean_age(census_dict, year):
    
    if year == 2001:
        column_names = ['LocalArea', 'Type', 'Total', '0 to 4 years', 
                   '5 to 9 years', '10 to 14 years', '15 to 19 years',
                   '20 to 24 years', '25 to 29 years', '30 to 34 years', 
                   '35 to 39 years', '40 to 44 years', '45 to 49 years', 
                   '50 to 54 years', '55 to 59 years', '60 to 64 years', 
                   '65 to 69 years', '70 to 74 years', '75 to 79 years', 
                   '80 to 84 years', '85 to 89 years', '90 to 94 years',
                   '95 to 99 years', '100 years and over']

        male = census_dict['Male']
        female = census_dict['Female']
    
        female.insert(1, 'Type', 'female')
        female.set_axis(column_names, axis=1, inplace=True)
        male.insert(1, 'Type', 'male')
        male.set_axis(column_names, axis=1, inplace=True)
    
        merged = pd.concat([female, male])
        merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
        total = merged.groupby('LocalArea').sum()
        total['Type'] = 'total'
        total.reset_index(inplace=True)
        merged = pd.concat([merged, total])
    
    else:
        if year == 2006:
            
            column_names = ['LocalArea', 'Type', 'Total', '0 to 4 years', 
                       '5 to 9 years', '10 to 14 years', '15 to 19 years',
                       '20 to 24 years', '25 to 29 years', '30 to 34 years', 
                       '35 to 39 years', '40 to 44 years', '45 to 49 years', 
                       '50 to 54 years', '55 to 59 years', '60 to 64 years', 
                        '65 to 69 years', '70 to 74 years', '75 to 79 years', 
                       '80 to 84 years', '85 to 89 years', '90 to 94 years',
                       '95 to 99 years', '100 years and over', 'Median Age']
            
            total = census_dict['Male & Female, Total']
            male = census_dict['Male, Total']
            female = census_dict['Female, Total']
        
        elif year == 2011:
            
            column_names = ['LocalArea', 'Type', 'Total', '0 to 4 years', 
                       '5 to 9 years', '10 to 14 years', '15 to 19 years',
                       '15 years', '16 years', '17 years', '18 years', '19 years',
                       '20 to 24 years', '25 to 29 years', '30 to 34 years', 
                       '35 to 39 years', '40 to 44 years', '45 to 49 years', 
                       '50 to 54 years', '55 to 59 years', '60 to 64 years', 
                       '65 to 69 years', '70 to 74 years', '75 to 79 years', 
                       '80 to 84 years', '85 years and over', 'Median age',
                       '% of the population aged 15 and over']
        
            total = census_dict['Total population by age groups']
            male = census_dict['Males, total']
            female = census_dict['Females, total']

        elif year == 2016:
            
            column_names = ['LocalArea', 'Type', 'Total', '0 to 14 years', 
                       '0 to 4 years', '5 to 9 years', '10 to 14 years',
                       '15 to 64 years', '15 to 19 years', '20 to 24 years', 
                       '25 to 29 years', '30 to 34 years', '35 to 39 years', 
                       '40 to 44 years', '45 to 49 years', '50 to 54 years', 
                       '55 to 59 years', '60 to 64 years', '65 years and over', 
                       '65 to 69 years', '70 to 74 years', '75 to 79 years', 
                       '80 to 84 years', '85 years and over','85 to 89 years', 
                       '90 to 94 years', '95 to 99 years', '100 years and over']
        
            total = census_dict['Total - Age groups and average age of the population - 100% data']
            male = census_dict['Total - Age groups and average age of males - 100% data']
            female = census_dict['Total - Age groups and average age of females - 100% data']
         
        female.insert(1, 'Type', 'female')
        female.set_axis(column_names, axis=1, inplace=True)
        male.insert(1, 'Type', 'male')
        male.set_axis(column_names, axis=1, inplace=True)
        total.insert(1, 'Type', 'total')
        total.set_axis(column_names, axis=1, inplace=True)
    
        merged = pd.concat([female, male, total])
    
    merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
    census_dict['population by age and sex'] = merged
    merged.to_csv('../../data/processed/census_population_age_sex_' + str(year) + '.csv')
    
    return census_dict

In [5]:
def clean_marital_status(census_dict, year):
    
    if year in [2001, 2006]:
        column_names = ['LocalArea', 'Total population 15 years and over',
                        'Single (never legally married)',
                       'Married', 'Separated', 'Divorced',
                       'Widowed', 'total x', 'Not living common law',
                       'Living common law']
        
        columns_ordered = ['LocalArea', 'Total population 15 years and over',
                       'Married or living with a or common-law partner',
                       'Married', 'Living common law',
                       'Not living with a married spouse or common-law partner',
                       'Single (never legally married)', 'Separated', 'Divorced',
                       'Widowed']
        
        df1 = census_dict['Total population 15 years and over by legal marital status']
        df2 = census_dict['Total population 15 years and over by common-law status']
        
        merged = pd.merge(df1, df2, on=['LocalArea'])
        merged.set_axis(column_names, axis=1, inplace=True)

        merged['Married or living with a or common-law partner'] = merged['Married'] + merged['Living common law']
        merged['Not living with a married spouse or common-law partner'] = merged['Total population 15 years and over'] - merged['Married or living with a or common-law partner']
        merged = merged[columns_ordered]
        
    
    else:
        if year == 2011:
            total = census_dict['Total population 15 years and over by marital status']
            male = census_dict['Males 15 years and over by marital status']
            female = census_dict['Females 15 years and over by marital status']
        elif year == 2016:
            total = census_dict['Total - Marital status for the population aged 15 years and over - 100% data']
            male = census_dict['Total - Marital status for males aged 15 years and over - 100% data']
            female = census_dict['Total - Marital status for females aged 15 years and over - 100% data']
            
        column_names = ['LocalArea', 'Type', 'Total population 15 years and over',
                       'Married or living with a or common-law partner',
                       'Married', 'Living common law',
                       'Not living with a married spouse or common-law partner',
                       'Single (never legally married)', 'Separated', 'Divorced',
                       'Widowed']
        
        female.insert(1, 'Type', 'female')
        female.set_axis(column_names, axis=1, inplace=True)
        male.insert(1, 'Type', 'male')
        male.set_axis(column_names, axis=1, inplace=True)
        total.insert(1, 'Type', 'total')
        total.set_axis(column_names, axis=1, inplace=True)
    
        merged = pd.concat([female, male, total])
        merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
        
    census_dict['marital status'] = merged   
    merged.to_csv('../../data/processed/census_marital_status_' + str(year) + '.csv')
    return census_dict

In [6]:
def clean_private_households(census_dict, year):
    if year == 2011:
        column_names = ['LocalArea', 'Type', 'Persons in private households',
                   'Persons not in census families', 'Living with relatives',
                   'Living with non-relatives only', 'Living alone',
                   'Number of census family persons']
        
        total = census_dict['Total number of persons in private households']
        male = census_dict['Number of males in private households']
        female = census_dict['Number of females in private households']
        total_65 = census_dict['Total number of persons aged 65 years and over in private households']
        male_65 = census_dict['Number of males aged 65 years and over in private households']
        female_65 = census_dict['Number of females aged 65 years and over in private households']
        
        female.insert(1, 'Type', 'female')
        female.set_axis(column_names, axis=1, inplace=True)
        male.insert(1, 'Type', 'male')
        male.set_axis(column_names, axis=1, inplace=True)
        total.insert(1, 'Type', 'total')
        total.set_axis(column_names, axis=1, inplace=True)
        
        female_65.insert(1, 'Type', '65+ female')
        female_65.set_axis(column_names, axis=1, inplace=True)
        male_65.insert(1, 'Type', '65+ male')
        male_65.set_axis(column_names, axis=1, inplace=True)
        total_65.insert(1, 'Type', '65+ total')
        total_65.set_axis(column_names, axis=1, inplace=True)
    
        merged = pd.concat([female, male, total, female_65, male_65, total_65])
        merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
        census_dict['private households - individuals'] = merged
        merged.to_csv('../../data/processed/census_private_households_' + str(year) + '.csv')

    return census_dict

In [7]:
def clean_couple_fam_structure(census_dict, year):
    
    column_names = ['LocalArea', 'Type', 'Total', 
                    'Without children at home',
                    'With children at home', '1 child', '2 children',
                    '3 or more children']

    if year == 2016:
        total = census_dict['Total - Couple census families in private households - 100% data']
        total.insert(1, 'Type', 'total couples')
        total.set_axis(column_names, axis=1, inplace=True)
    
        census_dict['couples - family structure'] = total
        total.to_csv('../../data/processed/census_couples_family_structure_' + str(year) + '.csv')
        
    else:
        if year in [2011, 2006]:
            married = census_dict['Total couple families by family structure and number of children'] 
            married = married[['LocalArea', 'Married couples',
                               'Without children at home',
                               'With children at home', '1 child', '2 children',
                               '3 or more children']]
            common_law = census_dict['Common-law couples']
            
        elif year == 2001:
            married = census_dict['Total couple families by family structure']
            married = married[['LocalArea','Married couples', 
                               'Without children at home', 'With children at home',
                               '1 child', '2 children', '3 or more children']]
            common_law = census_dict['Common-law couples']
            
        married.insert(1, 'Type', 'married couples')
        married.set_axis(column_names, axis=1, inplace=True)
        common_law.insert(1, 'Type', 'common-law couples')
        common_law.set_axis(column_names, axis=1, inplace=True)
        
        merged = pd.concat([married, common_law])
        total = merged.groupby('LocalArea').sum()
        total['Type'] = 'total couples'
        total.reset_index(inplace=True)
        merged = pd.concat([merged, total])
        
        merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
        census_dict['couples - family structure'] = merged
        merged.to_csv('../../data/processed/census_couples_family_structure_' + str(year) + '.csv')
    
    return census_dict
        

In [31]:
def clean_language_detailed(census_dict, year):
    
    if year == 2006:
        mt_total = census_dict['Total population by mother tongue']
        home_total = census_dict['Total population by language spoken most often at home']
        home_total = home_total.iloc[:,0:104].copy()
        work_total = census_dict['Total population 15 years and over who worked since January 1, 2005 by language used most often at work']
        
        mt_total.rename(columns={mt_total.columns[1]:'Total'}, inplace=True)
        mt_total.insert(1, 'Type', 'mother tongue - total')
        home_total.rename(columns={home_total.columns[1]:'Total'}, inplace=True)
        home_total.insert(1, 'Type', 'language most often spoken at home - total')
        work_total.rename(columns={work_total.columns[1]:'Total'}, inplace=True)
        work_total.insert(1, 'Type', 'language most often spoken at work - total')
        
        merged = pd.concat([mt_total, home_total, work_total])
    
    elif year == 2001:
        mt_total = census_dict['Total population by mother tongue']
        home_total = census_dict['Total population by home language']
        home_total = home_total.groupby(home_total.columns, axis=1).sum()
        
        mt_total.rename(columns={mt_total.columns[1]:'Total'}, inplace=True)
        mt_total.insert(1, 'Type', 'mother tongue - total')
        home_total.rename(columns={'Total population by home language':'Total'}, inplace=True)
        home_total.insert(1, 'Type', 'language most often spoken at home - total')
        
        merged = pd.concat([mt_total, home_total])
    
    else:
        if year == 2011:
            mt_total = census_dict['Detailed mother tongue - Total population excluding institutional residents']
            mt_male = census_dict['Detailed mother tongue - Males excluding institutional residents']
            mt_female = census_dict['Detailed mother tongue - Females excluding institutional residents']
        
            home_total = census_dict['Detailed language spoken most often at home - Total population excluding institutional residents']
            home_male = census_dict['Detailed language spoken most often at home - Males excluding institutional residents']
            home_female = census_dict['Detailed language spoken most often at home - Females excluding institutional residents']
        
            home2_total = census_dict['Detailed other language spoken regularly at home - Total population excluding institutional residents']
            home2_male = census_dict['Detailed other language spoken regularly at home - Males excluding institutional residents']
            home2_female = census_dict['Detailed other language spoken regularly at home - Females excluding institutional residents']
        
        elif year == 2016:
            mt_total = census_dict['Total - Mother tongue for the total population excluding institutional residents - 100% data']
            mt_male = census_dict['Total - Mother tongue for males excluding institutional residents - 100% data']
            mt_female = census_dict['Total - Mother tongue for females excluding institutional residents - 100% data']
        
            home_total = census_dict['Total - Language spoken most often at home for the total population excluding institutional residents - 100% data']
            home_male = census_dict['Total - Language spoken most often at home for males excluding institutional residents - 100% data']
            home_female = census_dict['Total - Language spoken most often at home for females excluding institutional residents - 100% data']
        
            home2_total = census_dict['Total - Other language(s) spoken regularly at home for the total population excluding institutional residents - 100% data']
            home2_male = census_dict['Total - Other language(s) spoken regularly at home for males excluding institutional residents - 100% data']
            home2_female = census_dict['Total - Other language(s) spoken regularly at home for females excluding institutional residents - 100% data']
        
        mt_female.rename(columns={mt_female.columns[1]:'Total'}, inplace=True)
        mt_female.insert(1, 'Type', 'mother tongue - female')
        mt_male.rename(columns={mt_male.columns[1]:'Total'}, inplace=True)
        mt_male.insert(1, 'Type', 'mother tongue - male')
        mt_total.rename(columns={mt_total.columns[1]:'Total'}, inplace=True)
        mt_total.insert(1, 'Type', 'mother tongue - total')
        
        home_female.rename(columns={home_female.columns[1]:'Total'}, inplace=True)
        home_female.insert(1, 'Type', 'language most often spoken at home - female')
        home_male.rename(columns={home_male.columns[1]:'Total'}, inplace=True)
        home_male.insert(1, 'Type', 'language most often spoken at home - male')
        home_total.rename(columns={home_total.columns[1]:'Total'}, inplace=True)
        home_total.insert(1, 'Type', 'language most often spoken at home - total')
        
        home2_female.rename(columns={home2_female.columns[1]:'Total'}, inplace=True)
        home2_female.insert(1, 'Type', 'other language spoken at home - female')
        home2_male.rename(columns={home2_male.columns[1]:'Total'}, inplace=True)
        home2_male.insert(1, 'Type', 'other language spoken at home - male')
        home2_total.rename(columns={home2_total.columns[1]:'Total'}, inplace=True)
        home2_total.insert(1, 'Type', 'other language spoken at home - total')
        
        merged = pd.concat([mt_female, mt_male, mt_total, home_female, home_male, home_total, home2_female, home2_male, home2_total])
    
    merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
    census_dict['detailed language'] = merged
    merged.to_csv('../../data/processed/census_detailed_language_' + str(year) + '.csv')
        
    return census_dict
    

In [9]:
def clean_official_language(census_dict, year):
    column_names = ['LocalArea', 'Type', 'Total',
                     'English', 'French', 'English and French',
                     'Neither English nor French']
    if year == 2016:
        known = census_dict['Total - Knowledge of official languages for the total population excluding institutional residents - 100% data']
        first = census_dict['Total - First official language spoken for the total population excluding institutional residents - 100% data']
    
    elif year == 2011:
        known = census_dict['Knowledge of official languages - Total population excluding institutional residents']
        first = census_dict['First official language spoken - Total population excluding institutional residents']
    
    elif year in [2001, 2006]:
        known = census_dict['Total population by knowledge of official languages']
        first = census_dict['Total population by first official language spoken']
            
        
    known.insert(1, 'Type', 'knowledge of official languages')
    known.set_axis(column_names, axis=1, inplace=True)
    first.insert(1, 'Type', 'first official language spoken')
    first.set_axis(column_names, axis=1, inplace=True)  
    
    merged = pd.concat([known, first])
    merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
    census_dict['official language'] = merged
    merged.to_csv('../../data/processed/census_official_language_' + str(year) + '.csv')
    
    return census_dict

In [10]:
def clean_structural_dwelling_type(census_dict, year):
          
    if year == 2006:
        column_names = ['LocalArea', 'Total',
       'Single-detached house', 'Semi-detached house', 'Row house',
       'Apartment, duplex',
       'Apartment, building that has five or more storeys']
        
        df = census_dict['Total number of occupied private dwellings by structural type of dwelling']
    
    elif year in [2001, 2011, 2016]:
        column_names = ['LocalArea', 'Total',
       'Single-detached house', 'Semi-detached house', 'Row house',
       'Apartment, detached duplex',
       'Apartment, building that has five or more storeys',
       'Apartment, building that has fewer than five storeys',
       'Other single-attached house', 'Movable dwelling']
        
        if year == 2001:
            df = census_dict['Total number of occupied private dwellings by structural type of dwelling']
            df = df.iloc[:,0:10].copy()
        
        elif year == 2011:
            df = census_dict['Total number of occupied private dwellings by structural type of dwelling']
            df = df[['LocalArea',
                    'Total number of occupied private dwellings by structural type of dwelling',
                    'Single-detached house', 'Semi-detached house', 'Row house',
                    'Apartment, duplex', 
                    'Apartment, building that has five or more storeys', 
                    'Apartment, building that has fewer than five storeys',
                    'Other single-attached house', 'Movable dwelling']].copy()
    
        elif year == 2016:
            df = census_dict['Total - Occupied private dwellings by structural type of dwelling - 100% data']
            df = df[['LocalArea',
            'Total - Occupied private dwellings by structural type of dwelling - 100% data',
           'Single-detached house', 'Semi-detached house', 'Row house',
           'Apartment or flat in a duplex',
           'Apartment in a building that has five or more storeys',
           'Apartment in a building that has fewer than five storeys',
           'Other single-attached house', 'Movable dwelling']].copy()
        
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['structual dwelling type'] = df
    df.to_csv('../../data/processed/census_structual_dwelling_type_' + str(year) + '.csv')
    
    return census_dict

In [11]:
def clean_household_type(census_dict, year):
    
    return census_dict

In [12]:
def clean_household_size(census_dict, year):
    if year == 2001:
        column_names = ['LocalArea', 'Total households',
       '1 person', '2 persons', '3 persons', '4 to 5 persons',
       '6 or more persons',
       'Average household size']
    
        df = census_dict['Total number of private households by household size']
        
    elif year in [2006, 2011]:
        
        column_names = ['LocalArea', 'Total households',
       '1 person', '2 persons', '3 persons', '4 to 5 persons',
       '6 or more persons', 'Number of persons in private households',
       'Average household size']
    
        df = census_dict['Total number of private households by household size']
        
    elif year == 2016:
        
        column_names = ['LocalArea', 'Total households',
       '1 person', '2 persons', '3 persons', '4 persons', '5 or more persons',
       'Number of persons in private households', 'Average household size']
        
        df = census_dict['Total - Private households by household size - 100% data']
   
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['household size'] = df
    df.to_csv('../../data/processed/census_household_size_' + str(year) + '.csv')
    
    return census_dict

In [120]:
def clean_census_family_children(census_dict, year):
    
    return census_dict

In [ ]:
def clean_lone_parent(census_dict, year):
    return census_dict

In [ ]:
def clean_aboriginal(census_dict, year):
    return census_dict

In [ ]:
def clean_immigration_age(census_dict, year):
    return census_dict

In [ ]:
def clean_citizenship(census_dict, year):
    return census_dict

In [ ]:
def clean_worker_class(census_dict, year):
    return census_dict

In [ ]:
def clean_education(census_dict, year):
    return census_dict

In [ ]:
def clean_ethnic_origin(census_dict, year):
    return census_dict

In [ ]:
def clean_time_worked(census_dict, year):
    return census_dict

In [ ]:
def clean_generation_status(census_dict, year):
    return census_dict

In [ ]:
def clean_household_char(census_dict, year):
    return census_dict

In [ ]:
def clean_immigration_period(census_dict, year):
    return census_dict

In [ ]:
def clean_birth_place(census_dict, year):
    return census_dict

In [ ]:
def clean_household_income(census_dict, year):
    return census_dict

In [ ]:
def clean_individual_income(census_dict, year):
    return census_dict

In [ ]:
def clean_industry(census_dict, year):
    return census_dict

In [ ]:
def clean_labour_force_status(census_dict, year):
    return census_dict

In [ ]:
def clean_commute_time(census_dict, year):
    return census_dict

In [ ]:
def clean_mobility(census_dict, year):
    return census_dict

In [ ]:
def clean_transportation_mode(census_dict, year):
    return census_dict

In [ ]:
def clean_occupation(census_dict, year):
    return census_dict

In [ ]:
def clean_workplace_status(census_dict, year):
    return census_dict

In [ ]:
def clean_religion(census_dict, year):
    return census_dict

In [ ]:
def clean_shelter_cost(census_dict, year):
    return census_dict

In [ ]:
def clean_visible_minority(census_dict, year):
    return census_dict

In [ ]:
def clean_work_activity(census_dict, year):
    return census_dict

In [32]:
df_2001 = clean_census('../../data/raw/census_2001.csv', 2001)
df_2006 = clean_census('../../data/raw/census_2006.csv', 2006)
df_2011 = clean_census('../../data/raw/census_2011.csv', 2011)
df_2016 = clean_census('../../data/raw/census_2016.csv', 2016)

In [33]:
for key in df_2011.keys():
    print(key)

Total population by age groups
Males, total
Females, total
Total population 15 years and over by marital status
Males 15 years and over by marital status
Females 15 years and over by marital status
Total number of persons in private households
Number of males in private households
Number of females in private households
Total number of persons aged 65 years and over in private households
Number of males aged 65 years and over in private households
Number of females aged 65 years and over in private households
Total number of census families in private households
Total couple families by family structure and number of children
Common-law couples
Total lone-parent families by sex of parent and number of children
Total children in census families in private households
Total number of private households by household type
Total number of private households by household size
Total number of occupied private dwellings by structural type of dwelling
Detailed mother tongue - Total population ex